In [18]:
import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

In [25]:
MODEL_NAME = 'model'

# Freeze the graph

input_graph_path = 'jgi/'+MODEL_NAME+'.pbtxt'
checkpoint_path = 'jgi/'+MODEL_NAME+'.ckpt'
input_saver_def_path = ""
input_binary = False
output_node_names = "FullyConnected_2/Softmax"
restore_op_name = "save/restore_all"
filename_tensor_name = "save/Const:0"
output_frozen_graph_name = 'jgi/frozen_'+MODEL_NAME+'.pb'
output_optimized_graph_name = 'jgi/optimized_'+MODEL_NAME+'.pb'
clear_devices = True


freeze_graph.freeze_graph(input_graph_path, input_saver_def_path,
                          input_binary, checkpoint_path, output_node_names,
                          restore_op_name, filename_tensor_name,
                          output_frozen_graph_name, clear_devices, "")

INFO:tensorflow:Restoring parameters from jgi/model.ckpt
INFO:tensorflow:Froze 6 variables.
Converted 6 variables to const ops.
20 ops in the final graph.


In [26]:
input_graph_def = tf.GraphDef()
with tf.gfile.Open(output_frozen_graph_name, "rb") as f:
    data = f.read()
    input_graph_def.ParseFromString(data)

output_graph_def = optimize_for_inference_lib.optimize_for_inference(
        input_graph_def,
        ["InputData/X"], # an array of the input node(s)
        ["FullyConnected_2/Softmax"], # an array of output nodes
        tf.float32.as_datatype_enum)

# Save the optimized graph

f = tf.gfile.FastGFile(output_optimized_graph_name, "w")
f.write(output_graph_def.SerializeToString())